In [19]:
from dotenv import load_dotenv
import os

load_dotenv()

print(os.getenv("LANGCHAIN_PROJECT"))

agent-book


In [ ]:

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0.0)
output = model.invoke("自己紹介をしてください。")
print(output)

content='はじめまして。私はAIアシスタントのChatGPTです。人工知能の分野で研究開発されたシステムで、皆さまの様々な質問や要望にお答えすることができます。\n私は人工知能ですが、人間のように感情を持っているわけではありません。ただ、皆さまとコミュニケーションを取りながら、できる限り丁寧で適切な回答を心がけています。\nどうぞよろしくお願いいたします。' additional_kwargs={} response_metadata={'id': 'msg_015UZGfdZ7M98iRS66bc9nRS', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 18, 'output_tokens': 152, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-haiku-20240307'} id='run--f8fc316b-33ff-4f45-a323-286b643c3643-0' usage_metadata={'input_tokens': 18, 'output_tokens': 152, 'total_tokens': 170, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}}


In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンといいます！"),
    AIMessage(content="こんにちは、ジョンさん！私はあなたのアシスタントです。どのようにお手伝いできますか？"),
    HumanMessage(content="私の名前がわかりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)


はい、ジョンさんと言っていただきました。私はあなたの名前を覚えています。


In [5]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは!"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

こんにちは!どうぞよろしくお願いいたします。何か質問やお手伝いできることはありますか?

In [7]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""以下の料理のレシピを教えてください
                                      
料理名: {dish}""")

prompt_value = prompt.invoke({"dish": "オムライス"})
print(prompt_value)

text='以下の料理のレシピを教えてください\n\n料理名: オムライス'


In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザが入力した料理のレシピを教えてください。"),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

messages=[SystemMessage(content='ユーザが入力した料理のレシピを教えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


In [10]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="こんにちは！私はジョンといいます！"),
            AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
        ],
        "input": "私の名前がわかりますか？",
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンといいます！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前がわかりますか？', additional_kwargs={}, response_metadata={})]


In [11]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("oshima/recipe")

prompt_value = prompt.invoke({"dish": "オムライス"})
print(prompt_value)

messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='オムライス', additional_kwargs={}, response_metadata={})]


In [12]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")    

In [13]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザが入力した料理のレシピを考えてください。\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(format_instructions=format_instructions)

In [26]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: human ===")
print(prompt_value.messages[1].content)

=== role: system ===
ユーザが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: human ===
カレー


In [27]:
ai_message = model.invoke(prompt_value)
print(ai_message.content)

Here is a JSON instance for a curry recipe:

{
  "ingredients": [
    "Onion",
    "Garlic",
    "Ginger",
    "Curry powder",
    "Cumin",
    "Coriander",
    "Turmeric",
    "Chicken or beef",
    "Potatoes",
    "Carrots",
    "Coconut milk",
    "Tomato paste",
    "Salt",
    "Pepper"
  ],
  "steps": [
    "Chop the onion, garlic, and ginger.",
    "Heat oil in a large pot and sauté the onion, garlic, and ginger until fragrant.",
    "Add the curry powder, cumin, coriander, and turmeric. Stir and cook for 1-2 minutes.",
    "Add the chicken or beef and brown it.",
    "Add the potatoes and carrots. Pour in the coconut milk and tomato paste.",
    "Season with salt and pepper to taste.",
    "Simmer the curry for 30-45 minutes, until the meat and vegetables are tender.",
    "Serve the curry over rice."
  ]
}


In [31]:
recipe = output_parser.invoke(ai_message.content.strip("Here is a JSON instance for a curry recipe:"))
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['Onion', 'Garlic', 'Ginger', 'Curry powder', 'Cumin', 'Coriander', 'Turmeric', 'Chicken or beef', 'Potatoes', 'Carrots', 'Coconut milk', 'Tomato paste', 'Salt', 'Pepper'] steps=['Chop the onion, garlic, and ginger.', 'Heat oil in a large pot and sauté the onion, garlic, and ginger until fragrant.', 'Add the curry powder, cumin, coriander, and turmeric. Stir and cook for 1-2 minutes.', 'Add the chicken or beef and brown it.', 'Add the potatoes and carrots. Pour in the coconut milk and tomato paste.', 'Season with salt and pepper to taste.', 'Simmer the curry for 30-45 minutes, until the meat and vegetables are tender.', 'Serve the curry over rice.']


In [32]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
output = output_parser.invoke(ai_message)
print(type(output))
print(output)

<class 'str'>
こんにちは。私はAIアシスタントです。


In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザが入力した料理のレシピを教えてください。"),
        ("human", "{dish}"),
    ]
)

chain = prompt | model

ai_message = chain.invoke({"dish": "オムライス"})
print(ai_message.content)

オムライスのレシピは以下のようになります。

【材料】
- 卵 3個
- 米 1カップ
- 玉ねぎ 1/2個
- ケチャップ 大さじ2
- 塩・こしょう 適量

【作り方】
1. 玉ねぎを細かく切る。
2. 卵を2つに分け、1つは溶き卵にする。もう1つは白身と黄身に分ける。
3. 米を炒めて、玉ねぎ、ケチャップ、塩こしょうを加えて炒める。
4. 炒めた米をオーブンシートなどに広げ、溶き卵を流し入れる。
5. 卵が固まってきたら、白身と黄身に分けた卵を中央に置く。
6. 包み込むように丸めて、皿に盛り付ける。
7. 好みでケチャップをかけて完成。

ポイントは、卵の固さと米の食感のバランスを取ることです。好みの具材を加えたり、ケチャップの量を調整するなど、アレンジも楽しめます。


In [35]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザが入力した料理のレシピを教えてください。\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

output_parser = PydanticOutputParser(pydantic_object=Recipe)

prompt_with_format_instructions = prompt.partial(format_instructions=output_parser.get_format_instructions())

chain = prompt_with_format_instructions | model | output_parser

recipe = chain.invoke({"dish": "オムライス"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['卵', '米', '玉ねぎ', 'ケチャップ', 'バター'] steps=['卵を溶いて、玉ねぎを炒める', '卵を流し入れて、ふわふわに仕上げる', 'ご飯を加えてよく混ぜる', 'ケチャップを加えて味付けする', 'オムレツの上にご飯を乗せる']


In [ ]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

418


In [39]:
from langchain_text_splitters import CharacterTextSplitter

teext_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)

docs = teext_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 6803, which is longer than the specified 1000
Created a chunk of size 3302, which is longer than the specified 1000
Created a chunk of size 1851, which is longer than the specified 1000
Created a chunk of size 1639, which is longer than the specified 1000
Created a chunk of size 9269, which is longer than the specified 1000
Created a chunk of size 2579, which is longer than the specified 1000
Created a chunk of size 17715, which is longer than the specified 1000
Created a chunk of size 1700, which is longer than the specified 1000
Created a chunk of size 1135, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1098, which is longer than the specified 1000
Created a chunk of size 1433, which is longer than the specified 1000
Created a chunk of size 1300, which is longer than the specified 1000
Created a chunk of size 1166, which is longer than the specified 1000
Created a chunk of 

1454


In [41]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# モデルの名前を指定 (ローカルにダウンロードされる)
# 日本語モデルの例: "intfloat/multilingual-e5-large" や "sonoisa/t5-japanese-v1.1-base" など
embedding_model_name = "intfloat/multilingual-e5-large"
model_kwargs = {'device': 'cpu'} # GPUがある場合は'cuda'を指定
encode_kwargs = {'normalize_embeddings': False} # 必要に応じて正規化するかどうか

hf_embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

text = "これはテスト用の文章です。"
embedding = hf_embeddings.embed_query(text)
print(embedding[:10]) # 埋め込みの一部を表示

texts = [
    "これは最初の文章です。",
    "これは2番目の文章です。"
]
embeddings = hf_embeddings.embed_documents(texts)
print(embeddings[0][:10])

/Users/takatorisatoshi/Desktop/playground/langchain-agent-book/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.030450331047177315, -0.008112591691315174, -0.013246072456240654, -0.062330130487680435, 0.03151698037981987, -0.045135218650102615, -0.002460327697917819, 0.09321574866771698, 0.05472954362630844, -0.022930661216378212]
[0.04152926430106163, -0.004461626056581736, 0.006429820321500301, -0.04747125506401062, 0.03172549232840538, -0.04668398201465607, -0.0019512787694111466, 0.07106544822454453, 0.046124741435050964, -0.020641732960939407]


In [42]:
from langchain_chroma import Chroma

db = Chroma.from_documents(
    documents=docs,
    embedding=hf_embeddings
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [43]:
retriever = db.as_retriever()

In [44]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

context_docs = retriever.invoke(query)
print(f"length of context_docs: {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


length of context_docs: 4
metadata = {'file_type': '.mdx', 'file_path': 'docs/docs/concepts/document_loaders.mdx', 'source': 'docs/docs/concepts/document_loaders.mdx', 'file_name': 'document_loaders.mdx'}
# Document loaders
<span data-heading-keywords="document loader,document loaders"></span>

:::info[Prerequisites]

* [Document loaders API reference](/docs/how_to/#document-loaders)
:::

Document loaders are designed to load document objects. LangChain has hundreds of integrations with various data sources to load data from: Slack, Notion, Google Drive, etc.

## Integrations

You can find available integrations on the [Document loaders integrations page](/docs/integrations/document_loaders/).

## Interface

Documents loaders implement the [BaseLoader interface](https://python.langchain.com/api_reference/core/document_loaders/langchain_core.document_loaders.base.BaseLoader.html).

Each DocumentLoader has its own specific parameters, but they can all be invoked in the same way with the 

In [46]:
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈:"""
{context}
"""                                                                                    

質問: {question}
''')

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)                  

output = chain.invoke(query)
print(output)

はい、AWSのS3からデータを読み込むためのDocument loaderがあります。

文脈から、以下の2つのDocument loaderが確認できます:

1. S3DirectoryLoader
2. S3FileLoader

これらのDocument loaderを使用することで、S3のディレクトリやファイルからデータを読み込むことができます。

具体的な使用例は以下のように記載されています:

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

また、それぞれの使用例へのリンクも提供されています:

- S3DirectoryLoader: `/docs/integrations/document_loaders/aws_s3_directory`
- S3FileLoader: `/docs/integrations/document_loaders/aws_s3_file`


In [47]:
output_parser = StrOutputParser()

cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [48]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください"),
        ("human", "{question}")
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [49]:
cot_summarize_chain = cot_chain | summarize_chain
cot_summarize_chain.invoke({"question": "10 + 2 * 3"})

'結論:\n10 + 2 * 3 = 16'

In [50]:
from langchain_core.runnables import RunnableLambda

def upper(text: str) -> str:
    return text.upper()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

cain = prompt | model | output_parser | RunnableLambda(upper)
output = cain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


In [51]:
optimisitic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザの入力に対して楽観的な意見をください。"),
        ("human", "{input}"),  
    ]
)
optimisitic_chain = optimisitic_prompt | model | output_parser

pessimisitic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザの入力に対して悲観的な意見をください。"),
        ("human", "{input}"),
    ]
)
pessimisitic_chain = pessimisitic_prompt | model | output_parser

In [52]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimisitic_option": optimisitic_chain,
        "pessimisitic_option": pessimisitic_chain,
    }
)

output = parallel_chain.invoke({"input": "生成AIの進化について"})
pprint.pprint(output)

{'optimisitic_option': '生成AIの進化については、確かに懸念もありますが、私は楽観的に捉えています。\n'
                       '\n'
                       '技術の進歩は常に両刃の剣であり、慎重に扱う必要があります。しかし、生成AIは人間の創造性を補完し、新しい可能性を切り開いてくれると信じています。\n'
                       '\n'
                       '適切な倫理的ガイドラインの下で、生成AIは私たちの生活をより豊かにし、社会課題の解決に貢献してくれるはずです。人間とAIが協調して、より良い未来を築いていけると期待しています。\n'
                       '\n'
                       '技術の発展には不安もありますが、私たち人間が主体的に関与し、生成AIを活用していけば、きっと素晴らしい未来が待っているはずです。前向きな姿勢を持ち続けましょう。',
 'pessimisitic_option': '生成AIの進化については、確かに懸念すべき点もあります。人工知能が人間の仕事を奪ったり、倫理的な問題を引き起こしたりする可能性は否定できません。また、偽情報の拡散や個人情報の悪用など、社会的な影響も危惧されます。技術の発展は必ずしも良いことばかりではなく、慎重に対応していく必要があるでしょう。私たちには、生成AIの脅威に備え、適切な規制や倫理的な枠組みを構築していく責任があります。楽観的に考えるのではなく、この問題の深刻さを認識し、対策を講じていくべきだと思います。'}


In [53]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。２つの意見をまとめてください。"),
        ("human", "楽観的意見: {optimisitic_option}\n悲観的意見: {pessimisitic_option}"),

    ]
)

synthesize_chain = (
    RunnableParallel(
        {
            "optimisitic_option": optimisitic_chain,
            "pessimisitic_option": pessimisitic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"input": "生成AIの進化について"})
pprint.pprint(output)

('両意見を総合すると、生成AIの進化については、楽観的な見方と悲観的な見方が存在することがわかります。\n'
 '\n'
 '楽観的な意見では、生成AIが人間の創造性を補完し、新しい可能性を切り開いてくれると期待されています。適切な倫理的ガイドラインの下で、生成AIが社会課題の解決に貢献し、人間の生活をより豊かにしてくれると考えられています。技術の発展には不安もありますが、人間が主体的に関与し、生成AIを適切に活用していけば、素晴らしい未来が待っているとの期待が示されています。\n'
 '\n'
 '一方、悲観的な意見では、生成AIが人間の仕事を奪ったり、倫理的な問題を引き起こしたりする可能性、偽情報の拡散や個人情報の悪用など、社会的な影響を危惧しています。技術の発展は必ずしも良いことばかりではなく、慎重に対応していく必要があると指摘されています。生成AIの脅威に備え、適切な規制や倫理的な枠組みを構築していく責任があると述べられています。\n'
 '\n'
 'つまり、生成AIの進化については、その可能性と課題が両方存在しており、慎重に対応しながら、適切な活用と規制のバランスを取っていくことが重要だと考えられます。')


In [54]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "AIの進化について"})
print(topic)

AIの進化について


In [56]:
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈:"""
{context}
"""                                                                                    

質問: {question}
''')


In [64]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

output = chain.invoke("東京の今日の天気は？")
print(output)

文脈から、東京の今日の天気は「曇のち雨」であることがわかります。


In [58]:
from tavily import TavilyClient

tavily_api_key = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=tavily_api_key)
response = tavily_client.search("Who is Leo Messi?")

print(response)

{'query': 'Who is Leo Messi?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Lionel Messi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'content': 'He is the most decorated player in the history of professional football having won 45 team trophies, including twelve Big Five "Big Five (association football)") league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals in a calendar year (91), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists "Assist (association football)") (192) in La Liga, most assists (18) and goal contributions (32) in the Copa América, most goal contributions (21) in the World Cup, most international appearances (193) and international goals (112) by a South American male, and the second-most in the latter category outright.', 'score': 0.78788257, 'raw_content': Non

In [65]:
chain = {
    "question": RunnablePassthrough(),
    "context": retriever,    
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("福岡の今日の天気は？")
print(output)

{'question': '福岡の今日の天気は？', 'context': [Document(metadata={'title': '福岡（福岡）の天気 - Yahoo!天気・災害', 'source': 'https://weather.yahoo.co.jp/weather/jp/40/8210.html', 'score': 0.86928266, 'images': []}, page_content='福岡（福岡） 5月31日(土) 23℃[+1] 16℃[-1] 時間 | 0-6 | 6-12 | 12-18 | 18-24 降水 | 0％ | 0％ | 0％ | 0％ 風：北の風海上でははじめ北の風やや強く 波：1.5メートル後1メートル 福岡地方の警報・注意報 今日の福岡県の熱中症情報 ほぼ安全 のどが渇く前に水分補給を 雨雲レーダー 5月30日 23時55分時点 6月1日(日) 26℃[+3] 14℃[-2] 時間 | 0-6 | 6-12 | 12-18 | 18-24 降水 | 0％ | 0％ | 0％ | 0％ 風：北の風 波：1メートル 市区町村の天気 週間天気 6/2（月） | 6/3（火） | 6/4（水） | 6/5（木） | 6/6（金） | 6/7（土） 24℃17℃100％ | 26℃19℃90％ | 29℃18℃10％ | 29℃18℃0％ | 29℃18℃10％ | 28℃19℃20％ 100％ 90％ 10％ 0％ 10％ 20％ 洗濯指数90 絶好の洗濯日和。バスタオルも速乾 傘指数0 傘はいりません 紫外線指数80 紫外線が強烈。サングラスで目の保護も 重ね着指数40 パーカーや薄手のニット一枚でOK アイス指数10 体が冷えないように要注意 洗濯指数100 絶好の洗濯日和。バスタオルも速乾 傘指数0 傘はいりません 紫外線指数70 紫外線が強烈。日中は日陰を利用して 重ね着指数30 半袖にシャツなど重ねて調節を アイス指数40 あま～いアイスクリームがうまい 5/30(金)19時 関東の南で低気圧が発達 土曜日は東日本、東北で激しい雨や暴風に警戒 天気ニュース - 来週にかけて気温差に注意 まもなく6月 梅雨入りのタイミングは？ 市区町村の天気 閉じる防災情報 気象災害 地震・津波 火山噴火 過去の災害を知る・災害

## Chapter6 Advanced RAG

In [21]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

418


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [22]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# モデルの名前を指定 (ローカルにダウンロードされる)
# 日本語モデルの例: "intfloat/multilingual-e5-large" や "sonoisa/t5-japanese-v1.1-base" など
embedding_model_name = "intfloat/multilingual-e5-large"
model_kwargs = {'device': 'cpu'} # GPUがある場合は'cuda'を指定
encode_kwargs = {'normalize_embeddings': False} # 必要に応じて正規化するかどうか

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [23]:
from langchain_chroma import Chroma

db = Chroma.from_documents(documents, embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [24]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0.0)

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈:"""
{context}
"""

質問: {question}
''')

retriver = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriver,
} | prompt | model | StrOutputParser()

chain.invoke("LangChainの概要を教えて")

'LangChainは以下のような概要を持つフレームワークです:\n\n- LLM (Large Language Model) を活用したアプリケーション開発を支援するフレームワークです。\n- LLMアプリケーションのライフサイクル全般を簡素化します。\n- 開発では、LangChainの提供するコンポーネントや第三者製のインテグレーションを使ってアプリケーションを構築できます。LangGraphを使えば、ストリーミングやヒューマンインザループをサポートする状態を持つエージェントを構築できます。\n- 本番運用では、LangSmithを使ってアプリケーションの検査、モニタリング、評価を行い、継続的な最適化とデプロイを行えます。\n- デプロイでは、LangGraphプラットフォームを使ってLLMアプリケーションをプロダクション向けのAPIやアシスタントに変換できます。\n\nLangChainは、LLMやエンベディングモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装し、多数のプロバイダとの統合を提供しています。'

In [15]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}                                                       
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriver,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは以下のような特徴を持つフレームワークです:\n\n1. **標準化されたコンポーネントインターフェース**: 言語モデルやベクトルストアなどの様々なコンポーネントを統一的なインターフェースで扱えるようにしています。これにより、プロバイダーを簡単に切り替えたり、コンポーネントを組み合わせて使うことができます。\n\n2. **オーケストレーション**: 複雑なアプリケーションを構築するために、LangGraphというオーケストレーションフレームワークを提供しています。LangGraphは、ステートフル性、ストリーミング、ヒューマンインザループなどの重要な機能を備えています。\n\n3. **可視化と評価**: LangSmithを通じて、アプリケーションの可視化と評価を行うことができます。これにより、アプリケーションの動作を理解し、迅速に改善することができます。\n\nLangChainは、大規模言語モデルを活用したアプリケーション開発を容易にするためのエコシステムを提供しています。標準化されたインターフェース、オーケストレーション、可視化と評価の機能により、開発者はより効率的にアプリケーションを構築できるようになります。'

In [16]:
from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリクエスト")

query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデーターベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")                                                           

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

multi_qury_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriver.map(),
} | prompt | model | StrOutputParser()

multi_qury_rag_chain.invoke("LangChainの概要を教えて")
    

'LangChainは以下のような特徴を持つフレームワークです:\n\n1. **標準化されたコンポーネントインターフェース**: LLMアプリケーションに必要な様々なコンポーネント(言語モデル、ベクトルストア等)には多様なAPIが存在しますが、LangChainはこれらに標準的なインターフェースを提供することで、プロバイダ間の切り替えや複数のコンポーネントの組み合わせが容易になります。\n\n2. **オーケストレーション**: LLMアプリケーションが複雑化するにつれ、複数のコンポーネントやモデルを効率的に組み合わせる必要性が高まっています。LangGraphはこのようなオーケストレーションを支援するライブラリで、複雑なコントロールフローや永続化、人間との対話などの機能を提供します。\n\n3. **可視性と評価**: 複雑なアプリケーションでは内部の動作を把握することが難しくなるため、LangSmithはアプリケーションのトレースや評価を支援します。これにより、プロンプトの最適化やLLMの選択など、開発の速度を制限する課題に対して迅速な解決が可能になります。\n\nLangChainは、これらの機能を通じてLLMアプリケーション開発の様々な段階を支援します。チュートリアル、ハウツーガイド、概念ガイド、APIリファレンスなど、豊富なドキュメンテーションも提供されています。また、LangGraphやLangSmithなどの関連ツールも用意されており、LLMアプリケーションの開発から本番運用まで一貫してサポートしています。'

In [17]:
from langchain_core.documents import Document

def reciprocal_rank_fusion(
        retriver_outputs: list[list[Document]],
        k: int = 60,
) -> list[Document]:
    # 各ドキュメントのコンテンツ(文字列)とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriver_outputs:
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) を計算してスコアに加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアが高い順にソートして上位のドキュメントを取得
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)
    return [content for content, _ in ranked]


In [18]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriver.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChainの概要を教えて")

'LangChainは以下のような特徴を持つフレームワークです:\n\n1. **標準化されたコンポーネントインターフェース**: 言語モデルやベクトルストアなどの様々なコンポーネントを統一的に扱えるようにしています。これにより、プロバイダ間の切り替えや、複数のコンポーネントを組み合わせて使うことが容易になります。\n\n2. **オーケストレーション**: 複雑なアプリケーションを構築する際に、複数のコンポーネントを効率的に連携させるための機能を提供しています。LangGraphはこのようなオーケストレーションを支援するライブラリです。\n\n3. **可観測性と評価**: 複雑なアプリケーションを開発する際に、アプリケーションの内部状態を把握し、評価することが重要です。LangSmithはこのような可観測性と評価を支援するプラットフォームです。\n\nLangChainは、大規模言語モデルを活用したアプリケーション開発を容易にするためのフレームワークです。標準化されたインターフェース、オーケストレーション、可観測性と評価の機能を提供することで、開発者がより効率的にアプリケーションを構築できるようサポートしています。'

In [29]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(
        model="rerank-multilingual-v3.0",
        top_n=top_n,
    )

    return cohere_reranker.compress_documents(documents=documents, query=question)

rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriver
    }
    | RunnablePassthrough.assign(context = rerank)
    | prompt
    | model
    | StrOutputParser()
)


rerank_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは以下のような概要を持つフレームワークです:\n\n- LLM (Large Language Model) アプリケーションの開発を支援するフレームワークです。\n- LLMアプリケーションのライフサイクル全般を簡素化します。\n  - 開発: LangChainのオープンソースコンポーネントや第三者製のインテグレーションを使ってアプリケーションを構築できます。LangGraphを使って、ストリーミングやヒューマンインザループをサポートする状態を持つエージェントを構築できます。\n  - 本番運用: LangSmithを使ってアプリケーションを検査、監視、評価し、継続的な最適化とデプロイを行えます。\n  - デプロイ: LangGraphアプリケーションをAPI/アシスタントとして本番運用できるようにします。\n\n- LLMやエンベディングモデル、ベクトルストアなどの標準インターフェースを提供し、多数のプロバイダとの統合を行っています。\n- 複数のオープンソースライブラリから構成されており、コアとなる`langchain-core`、プロバイダ統合パッケージ、`langchain`、コミュニティ製の統合パッケージ、`langgraph`などがあります。\n- チュートリアル、ハウツーガイド、概念ガイド、APIリファレンスなどの豊富なドキュメントを提供しています。\n- LangSmithやLangGraphなどの関連ツールも提供されています。'

In [30]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriver.with_config(
    {"run_name": "langchain_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [31]:
from enum import Enum

class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"

class RouterOutput(BaseModel):
    route: Route

route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
        route_prompt
        | model.with_structured_output(RouterOutput)
        | (lambda x: x.route)
)                                                                                      

In [32]:
def routed_retriver(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown retriever: {route}")

route_rag_chain = ({
    "question": RunnablePassthrough(),
    "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriver)
    | prompt
    | model
    | StrOutputParser()
)

In [33]:
route_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは以下のような概要を持つフレームワークです:\n\n- LLM (Large Language Model) を活用したアプリケーション開発を支援するフレームワークです。\n- LLMアプリケーションのライフサイクル全般を簡素化します。\n- 開発では、LangChainの提供するコンポーネントや第三者製のインテグレーションを使ってアプリケーションを構築できます。LangGraphを使えば、ストリーミングやヒューマンインザループをサポートする状態を持つエージェントを構築できます。\n- 本番運用では、LangSmithを使ってアプリケーションの検査、モニタリング、評価を行い、継続的な最適化とデプロイを行えます。\n- デプロイでは、LangGraphプラットフォームを使ってLLMアプリケーションをプロダクション向けのAPIやアシスタントに変換できます。\n\nLangChainは、LLMやエンベディングモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装し、多数のプロバイダとの統合を提供しています。'

In [34]:
route_rag_chain.invoke("東京の今日の天気は？")

'文脈から、東京の今日の天気は「曇のち雨」であることがわかります。'

In [36]:
from langchain_community.retrievers import BM25Retriever

chroma_retriver = retriver.with_config(
    {"run_name": "chroma_retriver"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [37]:
from langchain_core.runnables import RunnableParallel

hybrid_retriver = (
    RunnableParallel({
        "chroma_documents": chroma_retriver,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [38]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriver,
    } | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは以下のような概要を持つフレームワークです:\n\n- LLM (Large Language Model) アプリケーションの開発を支援するフレームワークです。\n- LLMアプリケーションのライフサイクル全般をサポートしています:\n  - 開発: LangChainの再利用可能なコンポーネントやサードパーティ製のインテグレーションを使ってアプリケーションを構築できます。LangGraphを使えば、ステートフルなエージェントを構築できます。\n  - 本番運用: LangSmithを使ってアプリケーションを検査、監視、評価し、継続的に最適化・デプロイできます。\n  - デプロイ: LangGraphプラットフォームを使えば、LLMアプリケーションをAPI/アシスタントとして本番デプロイできます。\n- LLMやエンベディングモデル、ベクトルストアなどの標準インターフェースを提供しており、多数のプロバイダとの統合が可能です。\n- アーキテクチャはコアライブラリ、プロバイダ別パッケージ、LangChainライブラリ、コミュニティパッケージ、LangGraphなどから構成されています。\n- チュートリアル、ハウツーガイド、概念ガイド、APIリファレンスなど、豊富なドキュメントが用意されています。\n- LangSmith、LangGraphなどの拡張ツールも用意されており、LLMアプリケーションの本番運用をサポートしています。\n\nつまり、LangChainは、LLMを活用したアプリケーション開発を容易にするための包括的なフレームワークと言えます。豊富な機能と拡張性を備えており、LLMアプリケーションの開発から本番運用までをサポートしています。'

## Chapter7 LangSmithを使ったR AGアプリケーションの評価

In [48]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

418


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [56]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]   
    document.metadata["headlines"] = []

In [52]:
from ragas.testset import TestsetGenerator
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.testset.synthesizers import (
    SingleHopSpecificQuerySynthesizer,
    MultiHopAbstractQuerySynthesizer,
    MultiHopSpecificQuerySynthesizer,
    default_query_distribution
)


generator_llm = LangchainLLMWrapper(langchain_llm=model)
generator_embeddings = LangchainEmbeddingsWrapper(embeddings=embeddings)

generator = TestsetGenerator.from_langchain(
    llm=generator_llm,
    embedding_model=generator_embeddings,
)

# カスタムの query_distribution
qd = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),   # simple 相当
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),  # multi_context 相当
    (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),  # reasoning 相当
]

testset = generator.generate_with_langchain_docs(
    documents=documents[:5],
    testset_size=4,
    query_distribution=qd
)

testset.to_pandas()

Applying HeadlineSplitter:   0%|          | 0/5 [00:00<?, ?it/s]          unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
Applying SummaryExtractor:  67%|██████▋   | 2/3 [02:18<01:02, 62.62s/it] unable to apply transformation: 'LangchainLLMWrapper' object has no attribute 'agenerate_prompt'
Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/3 [00:00<?, ?it/s]unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string, found '<class 'NoneType'>'
unable to apply transformation: node.property('summary') must be a string

ValueError: No nodes that satisfied the given filer. Try changing the filter.

In [57]:
from ragas.testset.graph import KnowledgeGraph, Node, NodeType

# 1. ドキュメントの中身と metadata を確認
for doc in documents:
    print(len(doc.page_content), doc.metadata)

# 2. Transform 適用後の KG 確認
test_kg = KnowledgeGraph()
# ノード追加コードと transforms 適用を同様に行う
print("ノード数:", len(test_kg.nodes))

56456 {'source': 'cookbook/sql_db_qa.mdx', 'file_path': 'cookbook/sql_db_qa.mdx', 'file_name': 'sql_db_qa.mdx', 'file_type': '.mdx', 'filename': 'cookbook/sql_db_qa.mdx', 'headlines': []}
6111 {'source': 'docs/docs/introduction.mdx', 'file_path': 'docs/docs/introduction.mdx', 'file_name': 'introduction.mdx', 'file_type': '.mdx', 'filename': 'docs/docs/introduction.mdx', 'headlines': []}
1513 {'source': 'docs/docs/people.mdx', 'file_path': 'docs/docs/people.mdx', 'file_name': 'people.mdx', 'file_type': '.mdx', 'filename': 'docs/docs/people.mdx', 'headlines': []}
1883 {'source': 'docs/docs/_templates/integration.mdx', 'file_path': 'docs/docs/_templates/integration.mdx', 'file_name': 'integration.mdx', 'file_type': '.mdx', 'filename': 'docs/docs/_templates/integration.mdx', 'headlines': []}
89937 {'source': 'docs/docs/additional_resources/arxiv_references.mdx', 'file_path': 'docs/docs/additional_resources/arxiv_references.mdx', 'file_name': 'arxiv_references.mdx', 'file_type': '.mdx', 'fi

In [ ]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name):
    client.delete_dataset(dataset_name)

dataset = client.create_dataset(dataset_name)

In [ ]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append({
        "question": testset_record.input.question,        
    })
    outputs.append({
        "contexts": testset_record.output.contexts,
        "ground_truth": testset_record.output.ground_truth,
    })
    metadatas.append({
        "source": testset_record.metadata[0]["source"],
        "evolution_type": testset_record.evolution_type,
    })